# Rumus Aliran Pipa (internal flow)

In [13]:
import math

# Fungsi untuk cetak besar variabel dan satuannya dengan singkat dan rapih
# Prettify print function for variables with magnitude and unit
def print_p(magnitude, unit=" ", explanation= " ", rounding=2):
    magnitude_pretty = round(magnitude,rounding)
    print(str(magnitude_pretty) +" "+unit+" "+explanation)

def print_px(variable, magnitude, unit=" ", explanation=" ",  rounding=2):
    magnitude_pretty = round(magnitude,rounding)
    print(variable+" = "+str(magnitude_pretty) +" "+unit+" "+explanation)
    
# Bagaimana menentukan tipe aliran pipa?
def tipe_aliran_pipa(Re):
    if Re<=2000:
        x = "Aliran Laminer"
    elif Re>= 4000:
        x = "Aliran Turbulen"
    elif (Re>2000 and Re<4000):
        x = "Aliran Transisi Laminer ke Turbulen"
    else:
        x = "Tidak masuk rentang perhitungan. Cek lagi perhitungan atau tipe data."
    print(x)
    print("")

# Bagaimana menentukan faktor friksi dari aliran pipa?:
def faktor_friksi_aliran_pipa(Re, eps_per_D=0):
    if Re<=2000:
        f=64/Re
    elif Re>= 4000:
        f_inv_sqrt = -1.8*math.log10((eps_per_D/3.7)**1.11+(6.9/Re))    # Rumus eksplisit Haaland
        f = f_inv_sqrt**(-2)
    elif (Re>2000 and Re<4000):
        # print("Aliran transisi laminar ke turbulen. ")
        f_lam=64/Re
        f_inv_sqrt = -1.8*math.log10((eps_per_D/3.7)**1.11+(6.9/Re))    # Rumus eksplisit Haaland
        f_turb = f_inv_sqrt**(-2)
        x = (Re-2000)/(4000-2000)      # x=0 dekat laminar, x=1 dekat turbulen
        f = (1-x)*f_lam + x*f_turb
    return f

# Bagian Perhitungan Aliran

In [14]:
## Hitung V diketahui hp dan z

In [15]:
"""
D_pipa = D_3inch
BOV_A Existing (z_el= 1.24 m, L_pipa=8.69 m, K_pipa = 6.2)
BOV_A Lurus    (z_el= 1.24 m, L_pipa=8.24 m, K_pipa = 3.5)
BOV_B Existing (z_el= 1.08 m, L_pipa=3.81 m, K_pipa = 3.8)
BOV_B Lurus    (z_el= 1.08 m, L_pipa=2.94 m, K_pipa = 2.6)
"""


z_el = 1.08                    # [m of oil]
L_pipa = 2.94                  #[meter]
K_pipa = 2.6

# Diamter pipa
D_3per4inch = 0.019                    #[meter]  (3/4 inch)
D_3inch     = 0.076                    # [meter]  (3 inch)
D_2inch     = 0.051

D_pipa = D_3inch                      #[meter]  



# Konstanta2
g = 9.8                               #[meter/s2]

# Sifat Fluida
rho_minyak_30degC = 0.8754          #[g/cm3]
mu_minyak_30degC = 128.42           #[g/cm3]
rho_minyak_20degC = 0.8815          #[g/cm3]
mu_minyak_20degC = 239.39           #[g/cm3]
rho_minyak = rho_minyak_30degC
mu_minyak  = mu_minyak_30degC
rho_minyak = rho_minyak/1000*1e6     #[kg/m3]
print_px("rho minyak", rho_minyak,"kg/m3") #[mPa.s]
mu_minyak = mu_minyak/1000           #[Pa.s]
print_px("mu_minyak", mu_minyak,"Pa.s")

# hPakt dari aktuator balik dan z_elevasi
P_akt = 0                              #[kgf/cm2]
P_akt = 98070*P_akt                    #[Pa]
hPakt= P_akt/(rho_minyak*g)            #[m of oil]
print_px("hPakt", hPakt,"m of oil") #[m of oil]
print_px("z_el", z_el, "m of oil") #[m of oil]

# Sifat Pipa Lurus D, Q, hitung luas A_pipa
D_3per4inch = 0.019                    #[meter]  (3/4 inch)
D_3inch     = 0.076                    # [meter]  (3 inch)
D_pipa = D_3inch                      #[meter]   
##############################################################

eps_pipa = 0.3/1000                   # [meter]
eps_per_D = eps_pipa/D_pipa        
A_pipa = 0.25*3.14159*D_pipa**2    #[m2]

# K dari sistem pipa


## iterasi 0 ###################################################

# Estimasi awal nilai f
# Hitung luas penampang pipa dan Kecepatan Pipa
f = 0.05

# Hitung ulang nilai V
v_pipa = (2*g*(hPakt+z_el)/(f*L_pipa/D_pipa+K_pipa+1))**(0.5)   # [m/s]
Q = A_pipa*v_pipa     # [m3/s]
Q = Q*1000*60         # [liter/min]

print("")
print("##### iterasi 0")
print_px("A_pipa", A_pipa, "m2", rounding=4)
print_px("v_pipa", v_pipa,"m/s")
print_px("Q", Q,"liter/min")

## iterasi 1 ###################################################

# Hitung bilangan Reynold
Re = rho_minyak*v_pipa*D_pipa/mu_minyak
print_px("Re",Re,"")
#tipe_aliran_pipa(Re)
f = faktor_friksi_aliran_pipa(Re, eps_per_D)
print_px("faktor friksi f",f,rounding=4)

# Hitung ulang nilai V
v_pipa = (2*g*(hPakt+z_el)/(f*L_pipa/D_pipa+K_pipa+1))**(0.5)   # [m/s]
Q = A_pipa*v_pipa     # [m3/s]
Q = Q*1000*60         # [liter/min]

print("")
print("##### iterasi 1")
print_px("A_pipa", A_pipa, "m2", rounding=4)
print_px("v_pipa", v_pipa,"m/s")
print_px("Q", Q,"liter/min")

## iterasi 2 ###################################################

# Hitung bilangan Reynold
Re = rho_minyak*v_pipa*D_pipa/mu_minyak
print_px("Re",Re,"")
#tipe_aliran_pipa(Re)
f = faktor_friksi_aliran_pipa(Re, eps_per_D)
print_px("faktor friksi f",f,rounding=4)

# Hitung ulang nilai V
v_pipa = (2*g*(hPakt+z_el)/(f*L_pipa/D_pipa+K_pipa+1))**(0.5)   # [m/s]
Q = A_pipa*v_pipa     # [m3/s]
Q = Q*1000*60         # [liter/min]

#print_px("A_pipa", A_pipa, "m2", rounding=4)
print("")
print("##### iterasi 2")
print_px("v_pipa", v_pipa,"m/s")
print_px("Q", Q,"liter/min")

## iterasi 3 ###################################################

# Hitung bilangan Reynold
Re = rho_minyak*v_pipa*D_pipa/mu_minyak
print_px("Re",Re,"")
#tipe_aliran_pipa(Re)
f = faktor_friksi_aliran_pipa(Re, eps_per_D)
print_px("faktor friksi f",f,rounding=4)

# Hitung ulang nilai V
v_pipa = (2*g*(hPakt+z_el)/(f*L_pipa/D_pipa+K_pipa+1))**(0.5)   # [m/s]
Q = A_pipa*v_pipa     # [m3/s]
Q = Q*1000*60         # [liter/min]

#print_px("A_pipa", A_pipa, "m2", rounding=4)
print("")
print("##### iterasi 3")
print_px("v_pipa", v_pipa,"m/s")
print_px("Q", Q,"liter/min")

## iterasi 4 ###################################################

# Hitung bilangan Reynold
Re = rho_minyak*v_pipa*D_pipa/mu_minyak
print_px("Re",Re,"")
#tipe_aliran_pipa(Re)
f = faktor_friksi_aliran_pipa(Re, eps_per_D)
print_px("faktor friksi f",f,rounding=4)

# Hitung ulang nilai V
v_pipa = (2*g*(hPakt+z_el)/(f*L_pipa/D_pipa+K_pipa+1))**(0.5)   # [m/s]
Q = A_pipa*v_pipa     # [m3/s]
Q = Q*1000*60         # [liter/min]

#print_px("A_pipa", A_pipa, "m2", rounding=4)
print("")
print("##### iterasi 4")
print_px("v_pipa", v_pipa,"m/s")
print_px("Q", Q,"liter/min")

## iterasi 5 ###################################################

# Hitung bilangan Reynold
Re = rho_minyak*v_pipa*D_pipa/mu_minyak
print_px("Re",Re,"")
#tipe_aliran_pipa(Re)
f = faktor_friksi_aliran_pipa(Re, eps_per_D)
print_px("faktor friksi f",f,rounding=4)

# Hitung ulang nilai V
v_pipa = (2*g*(hPakt+z_el)/(f*L_pipa/D_pipa+K_pipa+1))**(0.5)   # [m/s]
Q = A_pipa*v_pipa     # [m3/s]
Q = Q*1000*60         # [liter/min]

#print_px("A_pipa", A_pipa, "m2", rounding=4)
print("")
print("##### iterasi 5")
print_px("v_pipa", v_pipa,"m/s")
print_px("Q", Q,"liter/min")




rho minyak = 875.4 kg/m3  
mu_minyak = 0.13 Pa.s  
hPakt = 0.0 m of oil  
z_el = 1.08 m of oil  

##### iterasi 0
A_pipa = 0.0045 m2  
v_pipa = 1.96 m/s  
Q = 532.33 liter/min  
Re = 1013.21   
faktor friksi f = 0.0632    

##### iterasi 1
A_pipa = 0.0045 m2  
v_pipa = 1.87 m/s  
Q = 509.41 liter/min  
Re = 969.58   
faktor friksi f = 0.066    

##### iterasi 2
v_pipa = 1.85 m/s  
Q = 504.83 liter/min  
Re = 960.88   
faktor friksi f = 0.0666    

##### iterasi 3
v_pipa = 1.85 m/s  
Q = 503.89 liter/min  
Re = 959.08   
faktor friksi f = 0.0667    

##### iterasi 4
v_pipa = 1.85 m/s  
Q = 503.69 liter/min  
Re = 958.7   
faktor friksi f = 0.0668    

##### iterasi 5
v_pipa = 1.85 m/s  
Q = 503.65 liter/min  
